## BERT Based Sentiment Analysis Model Server
The model used here, was trained with the concept of transfer learning  i.e. taking huggingface transformers pretrained BERT model and further training it on a custom dataset of reviews. this yields a sentiment analysis model based on the prior knowledge of BERT. 
The model server is given a list of texts and outputs a list of labels corresponding to its prediction.
The labels express the sentiment of the writer towards the topic of the text:
0 for negative sentiment, 1 for neutral and 2 for positive.

The model file (~430 MB), can be downloaded to your local environment from: https://iguazio-sample-data.s3.amazonaws.com/models/model.pt

In [1]:
# nuclio: ignore
import nuclio

In [2]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from cloudpickle import dumps
import mlrun

### function code 
below is the model architecture, implemented with pytorch whose main component is bert.

In [3]:
PRETRAINED_MODEL = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

class BertSentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(BertSentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRETRAINED_MODEL)
        self.dropout = nn.Dropout(p=0.2)
        self.out_linear = nn.Linear(self.bert.config.hidden_size, n_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask):
        _, pooled_out = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        out = self.dropout(pooled_out)
        out = self.out_linear(out)
        return self.softmax(out)

#### serving interface
The load function essentialy instantiates our custom model with the architecture defined above.

In [4]:

class SentimentClassifierServing(mlrun.serving.V2ModelServer):
    def load(self):
        model_file, _ = self.get_model('.pt')
        device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
        model = BertSentimentClassifier(n_classes=3)
        model.load_state_dict(torch.load(model_file, map_location=device))
        model.eval()
        self.model = model
    def predict(self, body):
        try:
            instances = body['inputs']
            enc = tokenizer.batch_encode_plus(instances, return_tensors='pt', pad_to_max_length=True)
            outputs = self.model(input_ids=enc['input_ids'], attention_mask=enc['attention_mask'])
            _, preds = torch.max(outputs, dim=1)
            return preds.cpu().tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

In [5]:
# nuclio: end-code

### mlconfig

In [6]:
from mlrun import mlconf
import os

mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

### test locally
You may change model_dir to point at the path where model.pt file is saved

In [7]:
# Run this to download the pre-trained model to your `models` directory
import os
model_location = 'https://iguazio-sample-data.s3.amazonaws.com/models/model.pt'
saved_models_directory = os.path.join(os.path.abspath('./'), 'models')

# Create paths
os.makedirs(saved_models_directory, exist_ok=1)
model_filepath = os.path.join(saved_models_directory, os.path.basename(model_location))
!wget -nc -P {saved_models_directory} {model_location} 

File ‘/User/myfunctions/functions/sentiment_analysis_serving/models/model.pt’ already there; not retrieving.



In [8]:
import mlrun
models_path = model_filepath
fn = mlrun.code_to_function('my_server', kind='serving')
# set the topology/router and add models
graph = fn.set_topology("router")
fn.add_model("model1", class_name='SentimentClassifierServing', model_path=models_path)
# create and use the graph simulator
server = fn.to_mock_server()

> 2021-03-15 14:38:21,799 [info] model model1 was loaded
> 2021-03-15 14:38:21,800 [info] Loaded ['model1']


#### test 1
Here we test a pretty straightforward example for positive sentiment.

In [9]:
output = server.test("/v2/models/model1/infer", {"inputs":['I had a pleasure to work with such dedicated team. Looking forward to \
             cooperate with each and every one of them again.']})

assert output['outputs'] == [2]
print(output['outputs'])

[2]


#### test 2
Now we will test a couple more examples. These are arguably harder due to misleading words that express, on their own, an opposite sentiment comparing to the full text. 

In [10]:
output2 = server.test("/v2/models/model1/infer",{"inputs":['This app is amazingly useless.',
                     'As much as I hate to admit it, the new added feature is surprisingly user friendly.']})

print(output2['outputs'])
assert output['outputs'] == [2]
assert output2['outputs'] == [0,2]

[0, 2]


### remote activation
Create a function object with custom specification.

In [11]:
from mlrun import new_model_server, mount_v3io
import requests

In [12]:
from mlrun import code_to_function, mount_v3io
fn = code_to_function('sentiment-analysis-serving', kind='serving')
fn.spec.default_class = 'SentimentClassifierServing'
fn.apply(mount_v3io())
fn.spec.build.commands = ['pip install transformers==3.0.2']
fn.export("function.yaml")

> 2021-03-15 14:38:25,696 [info] function spec saved to path: function.yaml


In [13]:
fn.spec.base_spec['spec']['build']['baseImage']='mlrun/ml-models'
fn.add_model("model1", class_name='SentimentClassifierServing', model_path=models_path)
addr = fn.deploy(project='nlp-servers')


> 2021-03-15 14:38:25,710 [info] Starting remote function deploy
2021-03-15 14:38:26  (info) Deploying function
2021-03-15 14:38:26  (info) Building
2021-03-15 14:38:26  (info) Staging files and preparing base images
2021-03-15 14:38:26  (info) Building processor image
2021-03-15 14:38:27  (info) Build complete
2021-03-15 14:38:51  (info) Function deploy complete
> 2021-03-15 14:38:52,401 [info] function deployed, address=default-tenant.app.dev8.lab.iguazeng.com:31516


#### remote test
We will send a sentence to the model server via HTTP request. Note that the url below uses model server notation that directs our event to the predict function.

In [14]:
import json

event_data = {'inputs': ['I had a somewhat ok experience buying at that store.']}

resp = requests.put(addr + '/v2/models/model1/infer', json=json.dumps(event_data))

In [15]:
print(resp.text)

{"id": "fe026dda-2b2a-4444-ab2c-31046f977603", "model_name": "model1", "outputs": [1]}


The model server classified the sentence as neutral. 